In [ ]:
from __future__ import annotations

import re
from pathlib import Path

In [ ]:
equations = []

with Path("day07_input.txt").open() as file:
    for line in file:
        if numbers := re.findall(r"(\d+)", line):
            equations.append(list(map(int, numbers)))

I initially tried a brute force combinatorial approach, trying all possible combinations
of operators. But it is a bit slow, especially for the second part.

A more elegant idea is to start from the right end of the numbers:

- If the expected value is divisible by the last number, we guess that the last operator
  is multiplication. We divide the expected value by the last number and continue
  recursively with the rest of the numbers.
- If we can subtract the last number from the expected value, we guess that the last
  operator is addition. We subtract the last number from the expected value and continue
  with the rest of the numbers.

...and for Part 2:

- If the expected value ends with the last number, we guess that the last operator is
  concatenation. We remove the last number from the expected value and continue with the
  rest of the numbers.


In [ ]:
def is_solvable(expected: int, numbers: list[int], is_part2: bool) -> bool:
    """Check if the equation is solvable."""
    # Base recursive case: If there is only one number left in the list, it must be
    # equal to the expected value. Otherwise, this branch failed.
    if len(numbers) == 1:
        return numbers[0] == expected

    # Pop the last number from the list.
    last = numbers.pop()

    # Division / multiplication
    if (
        (last != 0)
        and (expected % last == 0)
        and is_solvable(expected // last, numbers[:], is_part2)
    ):
        return True

    # Subtraction / addition
    if (expected - last >= 0) and is_solvable(expected - last, numbers[:], is_part2):
        return True

    # Chopping / concatenation
    str_expected = str(expected)
    str_last = str(last)
    if (  # noqa: SIM103
        is_part2
        and len(str_expected) - len(str_last) > 0  # There must be something left.
        and str_expected.endswith(str_last)
        and is_solvable(int(str_expected.removesuffix(str_last)), numbers[:], is_part2)
    ):
        return True

    # None of the above worked, this equation is not solvable.
    return False

# Part 1


In [ ]:
%%time

solution = 0
for equation in equations:
    expected, *numbers = equation
    if is_solvable(expected, numbers, is_part2=False):
        solution += expected

solution

# Part 2


In [ ]:
%%time

solution = 0
for equation in equations:
    expected, *numbers = equation
    if is_solvable(expected, numbers, is_part2=True):
        solution += expected
solution